In [1]:
import numpy as np
from ultralytics import YOLO
from detections import Tracker, Detector, KeypointDetector
from calculations import TeamAffiliation, PlayerPosition
from utils import utils
from visualisation import Radar, VideoGenerator

In [2]:
SOURCE_PATH = r"C:\Users\Marcin\Videos\ganzgut.mp4"
DEST_PATH = r"C:\Users\Marcin\Videos\ganzgut_rgb.mp4"

In [3]:
model = YOLO("../training/runs/detect/train4/weights/best.pt")
model_keypoints = YOLO("../training/runs/pose/train8/weights/best.pt")
detector = Detector(model, 0.5)
keypoint_detector = KeypointDetector(model_keypoints)
tracker = Tracker(detector)
positions = PlayerPosition()

In [4]:
# images_path = r'data/SoccerNetGS/gamestate-2024/valid/SNGS-021/img1'
# frames = utils.load_frames_from_images(images_path)
frames = utils.load_frames_from_video(SOURCE_PATH)

In [5]:
tracks = tracker.track(frames)


0: 384x640 15 players, 49.5ms
Speed: 4.0ms preprocess, 49.5ms inference, 53.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 11.1ms
Speed: 2.0ms preprocess, 11.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 players, 11.1ms
Speed: 1.0ms preprocess, 11.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 players, 8.1ms
Speed: 1.0ms preprocess, 8.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 players, 9.1ms
Speed: 0.0ms preprocess, 9.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 players, 9.1ms
Speed: 0.0ms preprocess, 9.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 players, 9.1ms
Speed: 1.1ms preprocess, 9.1ms inference, 1.0ms postprocess per image at shape 

In [6]:
keypoints = keypoint_detector.detect(frames)


0: 384x640 1 pitch, 21.2ms
Speed: 2.0ms preprocess, 21.2ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 11.1ms
Speed: 2.0ms preprocess, 11.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 11.1ms
Speed: 1.0ms preprocess, 11.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pitch, 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


In [7]:
keypoints_list = keypoint_detector.fill_blank(frames)

In [8]:
player_positions = positions.get_players_position(keypoints_list, tracks)

In [9]:
teams = TeamAffiliation(tracks, frames)

In [10]:
shirt_colors, rgb_colors = teams.get_shirt_colors()

In [15]:
player_teams: list[np.array] = teams.get_rgb_teams()

In [23]:
team_colors = teams.get_average_team_color(player_teams,rgb_colors)

In [16]:
position_df = utils.parse_player_data(player_positions, tracks, player_teams)

In [17]:
video = teams.annotate_frames()

In [18]:
radar = Radar(position_df, teams.get_team_colors())

In [19]:
video_generator = VideoGenerator(video, radar)

In [20]:
video_generator.generate_video_with_radar(DEST_PATH, n_frames_back=45)

100%|██████████| 553/553 [00:56<00:00,  9.82it/s]

Video saved to C:\Users\Marcin\Videos\ganzgut_rgb.mp4
